In [166]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from matplotlib import dates
import plotly.graph_objs as go
import plotly.offline as pyoff

#import Keras
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [202]:
df = pd.read_excel('itService.xlsx')

df = df.drop(['YY', 'MM'], axis=1)
df['YYMM'] = df['YYMM'].astype(str)
df['YYMM'] = df['YYMM'] + "01"
df['YYMM'] = pd.to_datetime(df['YYMM'])
df

,YYMM,CNT
0,2013-05-01,14
1,2013-06-01,70
2,2013-07-01,25
3,2013-08-01,15
4,2013-09-01,13
...,...,...
99,2021-08-01,127
100,2021-09-01,107
101,2021-10-01,137
102,2021-11-01,143


In [168]:
#create a new dataframe to model the difference
df_diff = df.copy()
#add previous sales to the next row
df_diff['prev_CNT'] = df_diff['CNT'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['CNT'] - df_diff['prev_CNT'])
df_diff.head(10)


,YYMM,CNT,prev_CNT,diff
1,2013-06-01,70,14.0,56.0
2,2013-07-01,25,70.0,-45.0
3,2013-08-01,15,25.0,-10.0
4,2013-09-01,13,15.0,-2.0
5,2013-10-01,19,13.0,6.0
6,2013-11-01,19,19.0,0.0
7,2013-12-01,40,19.0,21.0
8,2014-01-01,126,40.0,86.0
9,2014-02-01,82,126.0,-44.0
10,2014-03-01,82,82.0,0.0


In [169]:
#df_diff = df_diff.drop(index=[1,2,3,4,5,6,7, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92], axis=0)
#df_diff = df_diff.drop(index=[103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92], axis=0)
#r-square 너무 낮아서 2021년 데이터로만 사용 
#r-square 퍼센트가 낮으면 종속변수와 도릭변수의 상관관계가 부족
#종속변인과 독립변인 사이에 상관관계가 높을수록 1에 가까워진다.
df_diff = df_diff[df['YYMM'].between('2020-10-01', '2021-12-01')]
#df_diff = df_diff.drop(index=[1,2,3,4,5,6,7, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92], axis=0)
#df_diff = df_diff.drop(index=[1,2,3,4,5,6,7], axis=0)
df_diff

C:\Users\admin\AppData\Local\Temp/ipykernel_26076/338216559.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,YYMM,CNT,prev_CNT,diff
89,2020-10-01,10,21.0,-11.0
90,2020-11-01,7,10.0,-3.0
91,2020-12-01,13,7.0,6.0
92,2021-01-01,72,13.0,59.0
93,2021-02-01,325,72.0,253.0
94,2021-03-01,362,325.0,37.0
95,2021-04-01,264,362.0,-98.0
96,2021-05-01,163,264.0,-101.0
97,2021-06-01,200,163.0,37.0
98,2021-07-01,180,200.0,-20.0


In [170]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['YYMM'],
        y=df_diff['CNT'],
    )
]
plot_layout = go.Layout(
        title='ITService Count'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [171]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['YYMM'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='ITService Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [172]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_CNT'],axis=1)
#adding lags
for inc in range(1,6):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [173]:
df_supervised.head(10)

,YYMM,CNT,diff,lag_1,lag_2,lag_3,lag_4,lag_5
0,2021-03-01,362,37.0,253.0,59.0,6.0,-3.0,-11.0
1,2021-04-01,264,-98.0,37.0,253.0,59.0,6.0,-3.0
2,2021-05-01,163,-101.0,-98.0,37.0,253.0,59.0,6.0
3,2021-06-01,200,37.0,-101.0,-98.0,37.0,253.0,59.0
4,2021-07-01,180,-20.0,37.0,-101.0,-98.0,37.0,253.0
5,2021-08-01,127,-53.0,-20.0,37.0,-101.0,-98.0,37.0
6,2021-09-01,107,-20.0,-53.0,-20.0,37.0,-101.0,-98.0
7,2021-10-01,137,30.0,-20.0,-53.0,-20.0,37.0,-101.0
8,2021-11-01,143,6.0,30.0,-20.0,-53.0,-20.0,37.0
9,2021-12-01,144,1.0,6.0,30.0,-20.0,-53.0,-20.0


In [174]:
# 변동 폭 확인 lag_1은 변동의 -0.010 %를 설명합니다
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

-0.0206287418324389


In [175]:
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.5315544295850183


In [176]:
# Define the regression formula
#model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12 ', data=df_supervised)
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)
#model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.8979956356434623


In [177]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['CNT','YYMM'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

In [201]:
df_model[0:-6].values

array([[  37.,  253.,   59.,    6.,   -3.,  -11.],
       [ -98.,   37.,  253.,   59.,    6.,   -3.],
       [-101.,  -98.,   37.,  253.,   59.,    6.],
       [  37., -101.,  -98.,   37.,  253.,   59.]])

In [179]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [180]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [181]:
X_train.shape

(4, 1, 5)

In [182]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=260, batch_size=1, verbose=1, shuffle=False)

Epoch 1/260
4/4 [==============================] - 1s 1ms/step - loss: 1.0425
Epoch 2/260
4/4 [==============================] - 0s 1ms/step - loss: 1.0219
Epoch 3/260
4/4 [==============================] - 0s 1ms/step - loss: 1.0042
Epoch 4/260
4/4 [==============================] - 0s 1ms/step - loss: 0.9865
Epoch 5/260
4/4 [==============================] - 0s 1ms/step - loss: 0.9687
Epoch 6/260
4/4 [==============================] - 0s 1ms/step - loss: 0.9510
Epoch 7/260
4/4 [==============================] - 0s 1ms/step - loss: 0.9333
Epoch 8/260
4/4 [==============================] - 0s 1ms/step - loss: 0.9157
Epoch 9/260
4/4 [==============================] - 0s 1ms/step - loss: 0.8982
Epoch 10/260
4/4 [==============================] - 0s 1ms/step - loss: 0.8808
Epoch 11/260
4/4 [==============================] - 0s 1ms/step - loss: 0.8634
Epoch 12/260
4/4 [==============================] - 0s 1ms/step - loss: 0.8461
Epoch 13/260
4/4 [==============================] - 0s 1ms/st

In [192]:
y_pred = model.predict(X_test,batch_size=1)
X_test

array([[[-0.22033898, -1.01709402, -1.84210526, -0.6875    ,
          6.54285714]],

       [[-0.54237288, -0.23076923, -1.86639676, -1.7421875 ,
          0.37142857]],

       [[-0.72881356, -0.55555556, -0.74898785, -1.765625  ,
         -3.48571429]],

       [[-0.54237288, -0.74358974, -1.21052632, -0.6875    ,
         -3.57142857]],

       [[-0.25988701, -0.55555556, -1.47773279, -1.1328125 ,
          0.37142857]],

       [[-0.39548023, -0.27065527, -1.21052632, -1.390625  ,
         -1.25714286]]])

In [184]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)
pred_test_set


[[ 1.33150899 -0.22033898 -1.01709402 -1.84210526 -0.6875      6.54285714]]
[[ 1.0516386  -0.54237288 -0.23076923 -1.86639676 -1.7421875   0.37142857]]
[[-0.15376028 -0.72881356 -0.55555556 -0.74898785 -1.765625   -3.48571429]]
[[-0.45782259 -0.54237288 -0.74358974 -1.21052632 -0.6875     -3.57142857]]
[[-0.36700934 -0.25988701 -0.55555556 -1.47773279 -1.1328125   0.37142857]]
[[-0.94316846 -0.39548023 -0.27065527 -1.21052632 -1.390625   -1.25714286]]


array([[ 1.33150899, -0.22033898, -1.01709402, -1.84210526, -0.6875    ,
         6.54285714],
       [ 1.0516386 , -0.54237288, -0.23076923, -1.86639676, -1.7421875 ,
         0.37142857],
       [-0.15376028, -0.72881356, -0.55555556, -0.74898785, -1.765625  ,
        -3.48571429],
       [-0.45782259, -0.54237288, -0.74358974, -1.21052632, -0.6875    ,
        -3.57142857],
       [-0.36700934, -0.25988701, -0.55555556, -1.47773279, -1.1328125 ,
         0.37142857],
       [-0.94316846, -0.39548023, -0.27065527, -1.21052632, -1.390625  ,
        -1.25714286]])

In [191]:
pred_test_set_inverted

array([[  59.87412059,   37.        , -101.        ,  -98.        ,
          37.        ,  253.        ],
       [  40.56306362,  -20.        ,   37.        , -101.        ,
         -98.        ,   37.        ],
       [ -42.60945961,  -53.        ,  -20.        ,   37.        ,
        -101.        ,  -98.        ],
       [ -63.58975878,  -20.        ,  -53.        ,  -20.        ,
          37.        , -101.        ],
       [ -57.32364458,   30.        ,  -20.        ,  -53.        ,
         -20.        ,   37.        ],
       [ -97.07862383,    6.        ,   30.        ,  -20.        ,
         -53.        ,  -20.        ]])

In [186]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df[-7:].YYMM)
act_sales = list(df[-7:].CNT)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_CNT'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['YYMM'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
#for multistep prediction, replace act_sales with the predicted sales

In [189]:
df_result
df_sales_pred = pd.merge(df,df_result,on='YYMM',how='left')
df_result


,pred_CNT,YYMM
0,259,2021-07-01
1,220,2021-08-01
2,84,2021-09-01
3,43,2021-10-01
4,79,2021-11-01
5,45,2021-12-01


In [188]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df,df_result,on='YYMM',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['YYMM'],
        y=df_sales_pred['CNT'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['YYMM'],
        y=df_sales_pred['pred_CNT'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)